In [2]:
!pip -q install langchain-groq 

In [2]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
load_dotenv()
#os.getenv('GROQ_API_KEY')

True

In [11]:
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)


In [12]:
respons = llm.invoke('Hello i am NDIAYE')
respons.content

"Hello NDIAYE, it's nice to meet you. Is there something I can help you with or would you like to chat?"

In [7]:
loader = WebBaseLoader('https://en.wikipedia.org/wiki/Large_language_model')

In [8]:
page_data = loader.load().pop().page_content

In [9]:
page_data

'\n\n\n\nLarge language model - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nHistory\n\n\n\n\n\n\n\n\n2\nDataset preprocessing\n\n\n\n\nToggle Dataset preprocessing subsection\n\n\n\n\n\n2.1\nTokenization\n\n\n\n\n\n\n2.1.1\nBPE\

In [14]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
res = chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

In [15]:
res.content

'Ich liebe das Programmieren.'

In [17]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `QUALIFICATIONS DE BASE`, `DESCRIPTION`, and `QUALIFICATIONS APPRÉCIÉES`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

In [18]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://amazon.jobs/fr/jobs/2857524/applied-scientist-ii-amazon")
page_data = loader.load().pop().page_content
print(page_data)

Applied Scientist II, Amazon - ID du poste : 2857524 | Amazon.jobs
Skip to main contentAccueilÉquipesSitesDomaines d'activitéMy careerMy applicationsMy profileAccount securitySettingsSign outRessourcesAménagements pour personnes en situation de handicapBenefitsDiversity, equity, and inclusionConseils pour réussir vos entretiensLeadership principlesWorking at AmazonFAQ×Applied Scientist II, AmazonIdentifiant du poste: 2857524 | Amazon.com Services LLCPostulez maintenantDESCRIPTIONWe’re working to improve shopping on Amazon using the conversational capabilities of LLMs, and are searching for pioneers who are passionate about technology, innovation, and customer experience, and are ready to make a lasting impact on the industry.In this role you will influence our team’s science and business strategy with your analyses. You will be expected to identify and solve ambiguous problems and science deficiencies, and to provide informed solutions based on state of the art machine learning researc

In [19]:
chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [20]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'QUALIFICATIONS DE BASE': ['3+ years of building models for business application experience',
  "PhD, or Master's degree and 4+ years of CS, CE, ML or related field experience",
  'Experience in patents or publications at top-tier peer-reviewed conferences or journals',
  'Experience programming in Java, C++, Python or related language',
  'Experience in any of the following areas: algorithms and data structures, parsing, numerical optimization, data mining, parallel and distributed computing, high-performance computing'],
 'DESCRIPTION': 'We’re working to improve shopping on Amazon using the conversational capabilities of LLMs, and are searching for pioneers who are passionate about technology, innovation, and customer experience, and are ready to make a lasting impact on the industry. In this role you will influence our team’s science and business strategy with your analyses. You will be expected to identify and solve ambiguous problems and science deficiencies, and to provide infor